In [1]:

# Set up the notebook
%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

# Load libraries
from FRVRS import nu, fu
import os.path as osp
import pandas as pd

In [8]:

import random

print()
action_ticks_list = [223819, 223819, 223819, 241120, 256019, 256924, 285654, 289814, 317108, 319906, 321245, 367706, 368149, 568501, 571875]
ordered_ints_list = [random.randint(1, 3) for _ in range(len(action_ticks_list))]
print(f'ordered_ints_list = {sorted(ordered_ints_list)}')
random.shuffle(ordered_ints_list)
print(f'disordered_ints_list = {ordered_ints_list}')


ordered_ints_list = [1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3]
disordered_ints_list = [1, 3, 2, 2, 3, 3, 3, 2, 3, 1, 2, 2, 3, 3, 3]


In [11]:

ordered_ints_list = [1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3]
disordered_ints_list = [1, 3, 2, 2, 3, 3, 3, 2, 3, 1, 2, 2, 3, 3, 3]
print("Kendall Tau Coefficient:", tau)

# Derive Kendall Tau Distance (number of swaps needed for perfect order)
# Note: In this case, we only care about the distance (number of disagreements)
# and not the p-value.
distance = (len(disordered_ints_list) * (len(disordered_ints_list) - 1)) / 2 - tau

print("Kendall Tau Distance:", int(distance))

Kendall Tau Coefficient: 0.0
Kendall Tau Distance: 105


In [17]:

kendalltau?

Signature:
kendalltau(
    x,
    y,
    initial_lexsort=None,
    nan_policy='propagate',
    method='auto',
    variant='b',
)
Docstring:
Calculate Kendall's tau, a correlation measure for ordinal data.

Kendall's tau is a measure of the correspondence between two rankings.
Values close to 1 indicate strong agreement, and values close to -1
indicate strong disagreement. This implements two variants of Kendall's
tau: tau-b (the default) and tau-c (also known as Stuart's tau-c). These
differ only in how they are normalized to lie within the range -1 to 1;
the hypothesis tests (their p-values) are identical. Kendall's original
tau-a is not implemented separately because both tau-b and tau-c reduce
to tau-a in the absence of ties.

Parameters
----------
x, y : array_like
    Arrays of rankings, of the same shape. If arrays are not 1-D, they
    will be flattened to 1-D.
initial_lexsort : bool, optional
    Unused (deprecated).
nan_policy : {'propagate', 'raise', 'omit'}, optional
    Def

In [10]:

def kendall_tau_distance(disordered_list, ordered_list):
    """
    Calculates the Kendall Tau Distance between two lists.

    Args:
        disordered_list: The list in its original order.
        ordered_list: The list in its sorted order.

    Returns:
        The Kendall Tau Distance between the two lists.
    """
    n = len(disordered_list)
    distance = 0
    for i in range(n):
        for j in range(i + 1, n):
            
            # Check if the order is different in both lists
            if (
                disordered_list[i] > disordered_list[j] and ordered_list[i] < ordered_list[j]
            ) or (
                disordered_list[i] < disordered_list[j] and ordered_list[i] > ordered_list[j]
            ):
                distance += 1
    
    return distance

distance = kendall_tau_distance(disordered_ints_list, ordered_ints_list)
print("Kendall Tau Distance:", distance)

Kendall Tau Distance: 20


In [13]:

def kendall_tau_distance(list1, list2):
    
    # Create a dictionary to store the ranks of elements in list1
    ranks = {v: i for i, v in enumerate(list1)}
    
    # Calculate the number of concordant and discordant pairs
    concordant_pairs = 0
    discordant_pairs = 0
    
    for i in range(len(list2)):
        for j in range(i+1, len(list2)):
            
            # Check if the order is the same in both lists
            if (ranks[list2[i]] < ranks[list2[j]]) != (i < j): discordant_pairs += 1
            else: concordant_pairs += 1
    
    # Calculate Kendall Tau Distance
    kendall_tau = (discordant_pairs - concordant_pairs) / (discordant_pairs + concordant_pairs)
    
    return kendall_tau

distance = kendall_tau_distance(ordered_ints_list, disordered_ints_list)
print("Kendall Tau Distance:", distance)

Kendall Tau Distance: 0.2


In [3]:

# load data frames
data_frames_list = nu.load_data_frames(
    first_responder_master_registry_df='', first_responder_master_registry_file_stats_df='', first_responder_master_registry_scene_stats_df=''
)
logs_df = data_frames_list['first_responder_master_registry_df']
file_stats_df = data_frames_list['first_responder_master_registry_file_stats_df']
scene_stats_df = data_frames_list['first_responder_master_registry_scene_stats_df']

Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/first_responder_master_registry_df.pkl.
Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/first_responder_master_registry_file_stats_df.pkl.
Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/first_responder_master_registry_scene_stats_df.pkl.


In [5]:

df = fu.get_elevens_data_frame(
    logs_df, file_stats_df, scene_stats_df,
    needed_columns=['scene_type', 'is_scene_aborted', 'is_a_one_triage_file', 'responder_category', 'responder_type'], patient_count_filter_fn=lambda x: x.patient_id.nunique() >= 1
)

In [13]:

from IPython.display import HTML

groupby_columns = ['patient_id', 'injury_id']
print()
HTML(df.groupby(groupby_columns).size().to_frame().rename(columns={0: 'record_count'}).sort_values(groupby_columns).reset_index(drop=False).to_html())

,patient_id,injury_id,record_count
0,Bob_0 Root,Face Shrapnel,294
1,Bob_0 Root,L Chest Collapse,295
2,Bob_0 Root,L Stomach Puncture,294
3,Bob_0 Root,L Thigh Laceration,294
4,Bob_0 Root,L Wrist Amputation,294
5,Bob_0 Root,R Shin Amputation,294
6,Bob_4 Root,R Palm Laceration,8
7,Bob_9 Root,Ear Bleed,49
8,Gary_1 Root,Face Shrapnel,318
9,Gary_3 Root,R Chest Collapse,560


In [14]:

print(
    f"The required procedures I have to work with are {nu.conjunctify_nouns(fu.injury_required_procedure_order)}, and the tool types I have to work with are"
    f" {nu.conjunctify_nouns(fu.tool_type_order)}. Which goes with which?"
)

The required procedures I have to work with are tourniquet, gauzePressure, decompress, woundpack, airway, and none, and the tool types I have to work with are Tourniquet, Gauze_Pack, Needle, Naso, Nasal Airway, and Gauze_Dressing. Which goes with which?


In [18]:

mask_series = (df.tool_type == 'Naso')
df[mask_series].sample(5).dropna(axis='columns', how='all').T

,21494,5154,5126,5238,25060
action_type,TOOL_APPLIED,TOOL_APPLIED,TOOL_APPLIED,TOOL_APPLIED,TOOL_APPLIED
action_tick,241523,646066,641526,673067,446772
event_time,2022-03-15 14:30:59,2022-03-15 09:36:00,2022-03-15 09:36:00,2022-03-15 09:37:00,2022-03-15 15:15:28
session_uuid,2d26ca7b-1dbb-42e4-9bbf-bf0d131e2e06,86366fd5-be94-43cc-a9e9-7c98609c0a66,86366fd5-be94-43cc-a9e9-7c98609c0a66,86366fd5-be94-43cc-a9e9-7c98609c0a66,27c2fa44-ba05-417c-ae95-317b06870dc9
tool_applied_type,Naso,Naso,Naso,Naso,Naso
tool_applied_attachment_point,tubeGuide (UnityEngine.GameObject),tubeGuide (UnityEngine.GameObject),tubeGuide (UnityEngine.GameObject),tubeGuide (UnityEngine.GameObject),tubeGuide (UnityEngine.GameObject)
tool_applied_tool_location,trumpet_container (UnityEngine.GameObject),trumpet_container (UnityEngine.GameObject),trumpet_container (UnityEngine.GameObject),trumpet_container (UnityEngine.GameObject),trumpet_container (UnityEngine.GameObject)
tool_applied_sender,Naso_Airway(Clone) (UnityEngine.GameObject),Naso_Airway(Clone) (UnityEngine.GameObject),Naso_Airway(Clone) (UnityEngine.GameObject),Naso_Airway(Clone) (UnityEngine.GameObject),Naso_Airway(Clone) (UnityEngine.GameObject)
tool_applied_attach_message,AppliedTrumpet,AppliedTrumpet,AppliedTrumpet,AppliedTrumpet,AppliedTrumpet
tool_applied_patient_id,Gary_1 Root,Gary_3 Root,Gary_3 Root,Gary_1 Root,Gary_1 Root


In [17]:

groupby_columns = ['tool_type', 'tool_applied_attach_message']
print()
HTML(df.groupby(groupby_columns).size().to_frame().rename(columns={0: 'record_count'}).sort_values(groupby_columns).reset_index(drop=False).to_html())

,tool_type,tool_applied_attach_message,record_count
0,Tourniquet,AppliedDecompNeedle,0
1,Tourniquet,AppliedDressingGauze,0
2,Tourniquet,AppliedPackingGauze,0
3,Tourniquet,AppliedTourniquet,1415
4,Tourniquet,AppliedTrumpet,0
5,Gauze_Pack,AppliedDecompNeedle,0
6,Gauze_Pack,AppliedDressingGauze,0
7,Gauze_Pack,AppliedPackingGauze,962
8,Gauze_Pack,AppliedTourniquet,0
9,Gauze_Pack,AppliedTrumpet,0


In [19]:

for (session_uuid, scene_id, patient_id, injury_id), injury_df in df.groupby(fu.injury_groupby_columns):
    break

In [20]:

sorted(injury_df.columns)

['action_tick', 'action_type', 'bag_access_location', 'bag_closed_location', 'event_time', 'injury_body_region', 'injury_id', 'injury_record_body_region', 'injury_record_id', 'injury_record_injury_injury_locator', 'injury_record_injury_treated', 'injury_record_injury_treated_with_wrong_treatment', 'injury_record_patient_id', 'injury_record_required_procedure', 'injury_record_severity', 'injury_required_procedure', 'injury_severity', 'injury_treated_body_region', 'injury_treated_id', 'injury_treated_injury_injury_locator', 'injury_treated_injury_treated', 'injury_treated_injury_treated_with_wrong_treatment', 'injury_treated_patient_id', 'injury_treated_required_procedure', 'injury_treated_severity', 'is_a_one_triage_file', 'is_scene_aborted', 'location_id', 'patient_breath', 'patient_demoted_breath', 'patient_demoted_health_level', 'patient_demoted_health_time_remaining', 'patient_demoted_hearing', 'patient_demoted_id', 'patient_demoted_mood', 'patient_demoted_pose', 'patient_demoted_po

In [23]:

mask_series = ~injury_df.patient_id.isnull()
patient_id = injury_df[mask_series].patient_id.tolist()[0]
mask_series = (df.patient_id == patient_id)
patient_df = df[mask_series]